In [1]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
from IPython import get_ipython


In [2]:
CATEGORIES = ["no", "yes"]

In [3]:
def prepare(filepath):
    IMG_SIZE = 100 
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [4]:
model = tf.keras.models.load_model("Trained.model")

W0814 10:22:28.749589 12232 deprecation.py:506] From C:\Users\Yugesh\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0814 10:22:28.768580 12232 deprecation.py:506] From C:\Users\Yugesh\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0814 10:22:28.770578 12232 deprecation.py:506] From C:\Users\Yugesh\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated

In [5]:
img = ('C:/Users/Yugesh/Dataset/Test/ no.jpeg')

In [6]:
prediction = model.predict([prepare(img)])

In [7]:
prediction
prediction[0][0]
print(CATEGORIES[int(prediction[0][0])])

no


In [8]:
prediction
if prediction == [[1.]]:
    print("TUMOR DETECTED")
    orig_img = cv2.imread(img,1) # 1 indicates color image
    # OpenCV uses BGR while Matplotlib uses RGB format
    # Display the color image with matplotlib
    plt.imshow(cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    print=("Input MRI")
    plt.show()
    gray_img = cv2.cvtColor( orig_img, cv2.COLOR_BGR2GRAY ) 
    plt.imshow(gray_img,cmap='gray')
    plt.axis('off')
    
    #median filtering to remove grainy noise
    median_filtered = cv2.medianBlur(gray_img, 5)
    plt.imshow(median_filtered,cmap='gray')
    plt.axis('off')
    plt.show()
    
    #edge detection using sobel
    Gx= np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Gy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    img_sobelx = cv2.Sobel(median_filtered,cv2.CV_8U,1,0,ksize=3)
    img_sobely = cv2.Sobel(median_filtered,cv2.CV_8U,0,1,ksize=3)
    
    #del f = Gx + Gy
    # Adding mask to the image
    img_sobel = img_sobelx + img_sobely+gray_img
    plt.imshow(img_sobel,cmap='gray')
    plt.axis('off')
    plt.show()
    
    # Set threshold and maxValue
    threshold = 50
    maxValue = 255
 
    # Threshold the pixel values
    th, thresh = cv2.threshold(img_sobel, threshold, maxValue, cv2.THRESH_OTSU)
    plt.imshow(thresh,cmap='gray')
    plt.axis('off')
    plt.show()
    
    # To remove any small white noises in the image using morphological opening. 
    # To determine foreground and background
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

    plt.imshow(opening,cmap='gray')
    plt.axis('off')
    plt.show()
    
    # Black region shows sure background area
    # Dilation increases object boundary to background.
    sure_bg = cv2.dilate(opening,kernel,iterations=3)

    plt.imshow(sure_bg,cmap='gray')
    plt.axis('off')
    plt.show()
    
    #  White region shows sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

    plt.imshow(sure_fg,cmap='gray')
    plt.axis('off')
    plt.show()
    
    # Identifying regions where we don't know whether foreground and background
    # Watershed algorithm
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)

    plt.imshow(unknown,cmap='gray')
    plt.axis('off')
    plt.show()
    
    #Watershed Segmentation
    contours, hierarchy = cv2.findContours(sure_fg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    # Creating a numpy array for markers and converting the image to 32 bit using dtype paramter
    marker = np.zeros((gray_img.shape[0], gray_img.shape[1]),dtype = np.int32)

    marker = np.int32(sure_fg) + np.int32(sure_bg)
    
    # Marker Labelling
    for id in range(len(contours)):
        cv2.drawContours(marker,contours,id,id+2, -1)

    marker = marker + 1

    marker[unknown==255] = 0

    copy_img = orig_img.copy()

    cv2.watershed(copy_img, marker)

    imgplt = plt.imshow(marker)

    plt.colorbar()
    plt.show()
    
    copy_img[marker==-1]=(0,0,255)
    cv2.imwrite('img.jpg',copy_img)
    plt.imshow(copy_img,cmap='gray')
    plt.axis('off')
    plt.show()
    
    #Opening by erosion
    # The basic purpose of the operation is to show only that part of the image having 
    # more intensity which has the tumor that is the part of the image forming our desired extraction.
    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    erosion = cv2.morphologyEx(median_filtered, cv2.MORPH_ERODE, kernel)
    plt.imshow(erosion,cmap='gray')
    plt.axis('off')
    plt.show()
    
    #Opening by dilation
    dilation = cv2.morphologyEx(erosion, cv2.MORPH_DILATE, kernel)
    plt.imshow(dilation,cmap='gray')
    plt.axis('off')
    plt.show()
    
else:
    print("NO TUMOR DETECTED")

    


NO TUMOR DETECTED


In [9]:
get_ipython().magic('reset -sf') 